In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

In [2]:
print('開始する都道府県コードを入力してください')
start_pref = input()
print(f'開始する都道府県コード : {start_pref}')

print('終了する都道府県コードを入力してください')
finish_pref = input()
print(f'終了する都道府県コー : {finish_pref}')


#工事費用
cost_df = DataFrame(pd.read_csv('G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data/prefecture_cost.csv', encoding='shift_jis'))
pref = ['北海道',
        '青森',
        '岩手',
        '宮城',
        '秋田',
        '山形',
        '福島',
        '茨城',
        '栃木',
        '群馬',
        '埼玉',
        '千葉',
        '東京都',
        '神奈川',
        '新潟',
        '富山',
        '石川',
        '福井',
        '山梨',
        '長野',
        '岐阜',
        '静岡',
        '愛知',
        '三重',
        '滋賀',
        '京都',
        '大阪',
        '兵庫',
        '奈良',
        '和歌山',
        '鳥取',
        '島根',
        '岡山',
        '広島',
        '山口',
        '徳島',
        '香川',
        '愛媛',
        '高知',
        '福岡',
        '佐賀',
        '長崎',
        '熊本',
        '大分',
        '宮崎',
        '鹿児島',
        '沖縄']
pref_df = DataFrame(pref, columns=['prefecture'])
cost_df = pd.merge(pref_df, cost_df, on='prefecture')
#損害額の計算
def dmg_value(cost, total_area, dpc_rate, age, dmg_rate):
        #建物の金額
        house_price = cost * total_area
        #償却された金額
        dpc_price = house_price * 0.9 * dpc_rate * age
        #時価総額
        tmp_price = house_price - dpc_price
        #築年が多いため時価総額がマイナスになった場合の条件分岐
        if (tmp_price / house_price) < 0.05:
                tmp_price = house_price * 0.05
        #損害額
        dmg = tmp_price * dmg_rate
        return dmg


def house_price(cost, total_area):
        #減価償却なしの住宅の金額
        house_price = cost * total_area
        return house_price

%cd "G:/マイドライブ/akiyamalab/卒論/負担率総集編"
n = int(start_pref)
while n <= int(finish_pref):
    damage_path = f'./data/損害額data/{n}_step8_1.csv'
    df = pd.read_csv(damage_path)
    dmg_price_list = []

    #工事費用
    df['cost'] = np.where(df['estimated_structure'] == 0, cost_df['wood'][n] * 1000, cost_df['steel'][n] * 1000)
    #償却率
    df['dpc_rate'] = np.where(df['estimated_structure'] == 0, 0.031, 0.036)
    #築年数
    df['age'] = df['estimated_age'].map({1: 5, 2: 8, 3: 15.5, 4: 25.5, 5: 35.5, 6: 45.5, 7: 50})
    
    df['dmg_rate'] = np.where((df['A31_001'] == 11) | (df['A31_001'] == 21), np.where(df['floor'] == 0, 0.4, 0.35),
                          np.where((df['A31_001'] == 12) | (df['A31_001'] == 22), np.where(df['floor'] == 0, 0.6, 0.45),
                          np.where((df['A31_001'] == 13) | (df['A31_001'] == 23), np.where(df['floor'] == 0, 0.75, 0.5),
                          np.where(((df['A31_001'].between(14, 15)) | (df['A31_001'] >= 24)), np.where(df['floor'] == 0, 0.8, 0.55), 0.0))))
    
    #減価償却なしの住宅の価値
    df['house_price'] = df.apply(lambda row: house_price(row['cost'], row['total_area']), axis=1)
    
    #建物の損害額の計算
    df['damage_price'] = df.apply(lambda row: dmg_value(row['cost'], row['total_area'], row['dpc_rate'], row['age'], row['dmg_rate']), axis=1)
    
    
    damage_output_path = f'./dev/損害額/{n}_step8_1_output.csv'
    df.to_csv(damage_output_path, index=False)
    n += 1

開始する都道府県コードを入力してください
開始する都道府県コード : 1
終了する都道府県コードを入力してください
終了する都道府県コー : 47
G:\マイドライブ\akiyamalab\卒論\負担率総集編


c:\Users\imai_14\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


KeyError: 47

In [5]:
df.groupby('KEY_CODE', as_index=False)['damage_price'].sum()

,KEY_CODE,damage_price
0,141010180,9.604426e+08
1,141010490,2.556908e+09
2,141010560,1.623599e+09
3,141010570,1.237905e+09
4,141010580,1.910518e+09
...,...,...
1360,14366009001,2.109556e+08
1361,14366009002,5.511168e+05
1362,14366009003,9.168066e+07
1363,14366009004,9.471514e+07


In [13]:
df['KEY_CODE'].value_counts().sum()

265446

In [9]:
df

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,A31_001,cost,dpc_rate,age,dmg_rate,house_price,damage_price
0,1573,133.797213,34.672247,0,166.805,1364,166.805,331012270,1,4,12,256000,0.036,25.5,0.60,42702080.0,4.452973e+06
1,1574,133.797034,34.671509,0,131.640,1364,131.640,331012270,1,2,12,256000,0.036,8.0,0.60,33699840.0,1.497890e+07
2,1695,133.797357,34.672400,0,96.400,1364,96.400,331012270,1,3,12,256000,0.036,15.5,0.60,24678400.0,7.370945e+06
3,1688,133.797788,34.673460,0,184.710,1364,184.710,331012270,1,3,11,256000,0.036,15.5,0.40,47285760.0,9.415541e+06
4,1694,133.797064,34.672907,0,102.940,1364,102.940,331012270,1,1,11,256000,0.036,5.0,0.40,26352640.0,8.833405e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333097,19877,133.817980,34.858477,0,253.700,1364,253.700,33681059005,1,6,12,256000,0.036,45.5,0.60,64947200.0,1.948416e+06
333098,19890,133.816148,34.860425,0,107.045,1364,107.045,33681059005,1,3,11,256000,0.036,15.5,0.40,27403520.0,5.456589e+06
333099,19898,133.817725,34.860710,0,118.770,1364,118.770,33681059006,1,1,11,256000,0.036,5.0,0.40,30405120.0,1.019180e+07
333100,19919,133.818189,34.857995,0,162.060,1364,162.060,33681059005,1,4,13,256000,0.036,25.5,0.75,41487360.0,5.407877e+06


In [11]:
#工事費用
cost_df = DataFrame(pd.read_csv('G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data/prefecture_cost.csv', encoding='shift_jis'))
pref = ['北海道',
        '青森',
        '岩手',
        '宮城',
        '秋田',
        '山形',
        '福島',
        '茨城',
        '栃木',
        '群馬',
        '埼玉',
        '千葉',
        '東京都',
        '神奈川',
        '新潟',
        '富山',
        '石川',
        '福井',
        '山梨',
        '長野',
        '岐阜',
        '静岡',
        '愛知',
        '三重',
        '滋賀',
        '京都',
        '大阪',
        '兵庫',
        '奈良',
        '和歌山',
        '鳥取',
        '島根',
        '岡山',
        '広島',
        '山口',
        '徳島',
        '香川',
        '愛媛',
        '高知',
        '福岡',
        '佐賀',
        '長崎',
        '熊本',
        '大分',
        '宮崎',
        '鹿児島',
        '沖縄']
pref_df = DataFrame(pref, columns=['prefecture'])
cost_df = pd.merge(pref_df, cost_df, on='prefecture')

In [20]:
pref_list = [i for i in range(1, 48)]
cost_df['pref_code'] = pref_list
cost_df = cost_df[['pref_code', 'prefecture', 'wood', 'steel']]

cost_df.to_csv('G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/損害額data/prefecture_cost_2.csv', encoding='shift_jis', index=False)

In [21]:
#工事費用
df = pd.read_csv('G:/マイドライブ/akiyamalab/卒論/負担率総集編/data/住宅地図兼マイクロ人口/20_step10.csv', encoding='shift_jis')

In [25]:
df['assets'] = np.where(
    df['spouse_sex'] == 0, 3000000,
    np.where(
        df['household_age'] <= 29, 5000000,
        np.where(
            (df['household_age'] >= 30) & (df['household_age'] <= 39), 8000000,
            np.where(
                (df['household_age'] >= 40) & (df['household_age'] <= 49), 11000000,
                np.where(df['household_age'] >= 50, 11500000, 0)  # 最後の条件とデフォルト値
            )
        )
    )
)


In [46]:
df.iloc[:, 15:59]

,m_age0,m_age5,m_age10,m_age15,f_age0,f_age5,f_age10,f_age15,m_age20,m_age25,...,f_age60,f_age65,f_age70,f_age75,f_age80,f_age85,f_age90,f_age95,f_age100,f_age105
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110782,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
110783,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
110784,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
110785,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [58]:
df.columns

Index(['tatemon_id', 'x', 'y', 'floor', 'AREA', 'atype2', 'total_area',
       'KEY_CODE', 'estimated_structure', 'estimated_age', 'ave_saving',
       'household_age', 'household_sex', 'spouse_age', 'spouse_sex', 'm_age0',
       'm_age5', 'm_age10', 'm_age15', 'f_age0', 'f_age5', 'f_age10',
       'f_age15', 'm_age20', 'm_age25', 'm_age30', 'm_age35', 'm_age40',
       'm_age45', 'm_age50', 'm_age55', 'm_age60', 'm_age65', 'm_age70',
       'm_age75', 'm_age80', 'm_age85', 'm_age90', 'm_age95', 'm_age100',
       'm_age105', 'f_age20', 'f_age25', 'f_age30', 'f_age35', 'f_age40',
       'f_age45', 'f_age50', 'f_age55', 'f_age60', 'f_age65', 'f_age70',
       'f_age75', 'f_age80', 'f_age85', 'f_age90', 'f_age95', 'f_age100',
       'f_age105', 'num_child', 'num_adult', 'meshcode', ' T30_P03_SI',
       ' T30_P06_SI', ' T50_P02_SI', ' T50_P05_SI', ' T50_P10_SI',
       ' T50_P39_SI', 'assets', 'families'],
      dtype='object')

In [63]:
# DataFrameからNumPy配列に変換
data_children = df.iloc[:, 15:22].to_numpy()
data_adult = df.iloc[:, 23:59].to_numpy()

# NumPy配列の合計を計算
df['children'] = np.sum(data_children, axis=1)
df['adult'] = np.sum(data_adult, axis=1)

In [64]:
df['add_assets'] = df['children'] * 1300000 + df['adult'] * 800000

In [65]:
df

,tatemon_id,x,y,floor,AREA,atype2,total_area,KEY_CODE,estimated_structure,estimated_age,...,T50_P02_SI,T50_P05_SI,T50_P10_SI,T50_P39_SI,assets,families,children,data_adult,adult,add_assets
0,4,138.026349,36.564040,0,133.35,1364,133.35,202011790,1,1,...,5.8,5.5,5.3,4.8,3000000,3,0,3,3,2400000
1,9,138.026478,36.564145,0,142.50,1364,142.50,202011790,1,6,...,5.8,5.5,5.3,4.8,11500000,2,0,2,2,1600000
2,10,138.026516,36.563883,0,135.20,1364,135.20,202011790,1,2,...,5.8,5.5,5.3,4.8,11500000,3,0,3,3,2400000
3,18,138.029484,36.539159,0,371.04,1364,371.04,202011780,1,1,...,6.1,5.9,5.7,5.1,3000000,1,0,1,1,800000
4,5,138.035916,36.544212,0,378.78,1364,378.78,202011770,1,4,...,6.1,5.9,5.7,5.1,3000000,1,0,1,1,800000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110782,645,137.826096,35.521322,0,67.02,1364,67.02,20205506002,1,5,...,6.2,6.0,5.9,5.6,11500000,7,1,6,6,6100000
110783,652,137.826744,35.520677,0,83.03,1364,83.03,20205505002,1,2,...,6.2,6.0,5.9,5.6,3000000,2,0,1,1,800000
110784,653,137.826350,35.520731,0,49.42,1364,49.42,20205505002,1,4,...,6.2,6.0,5.9,5.6,11500000,4,0,3,3,2400000
110785,654,137.826560,35.520765,0,59.43,1364,59.43,20205505002,1,7,...,6.2,6.0,5.9,5.6,3000000,4,1,3,3,3700000


In [16]:
if row['spouse_sex'] == 0:
    assets = 3000000
    
else:
    if household_age <= 29:
        assets = 5000000
    
    elif 30 <= household_age <= 39:
        assets = 8000000
    
    elif 40 <= household_age <= 49:
        assets = 11000000
    
    elif 50 <= household_age:
        assets = 11500000

,pref_code,prefecture,wood,steel
0,1,北海道,187,256
1,2,青森,175,256
2,3,岩手,182,256
3,4,宮城,173,256
4,5,秋田,175,256
5,6,山形,180,256
6,7,福島,178,256
7,8,茨城,173,256
8,9,栃木,173,256
9,10,群馬,173,256
